In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import os
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
from itertools import product
import time
from itertools import product, combinations

### utility func

In [2]:
def save_pred_to_csv(pred, msg='', dir_name='../predictions/'):
    """
    save prediction results to csv.
    
    Add time as idfier. Other info can be passed to msg.
    
    Args:
        pred(DataFrame): prediction result.
        
        msg(str): extra message want to include in file_name
        
        dir(name): directory name
        
    Return None
    """
    try:
        pred = pred[['ID', 'item_cnt_month']]
    except:
        raise Exception('ID or item_cnt_month column not found')
    
    str_datetime = datetime.now().strftime("%m%d_%H_%M")
    full_name = dir_name + msg + '_' + str_datetime + '.csv'
    
    pred.to_csv(full_name, index=False)
    

In [3]:
data_dir = os.path.dirname(os.getcwd()) + '/data/'
files = os.listdir(data_dir)

train = pd.read_csv(data_dir + files[1])
test = pd.read_csv(data_dir + files[3])

shops = pd.read_csv(data_dir + files[2])
item_cat = pd.read_csv(data_dir + files[4])
item = pd.read_csv(data_dir + files[5])
sample_sub = pd.read_csv(data_dir + files[6])

### Data cleaning

In [4]:
# from https://www.kaggle.com/dlarionov/feature-engineering-xgboost

##### fix shop ids (duplicate shops)#####

# Якутск Орджоникидзе, 56
train.loc[train.shop_id == 0, 'shop_id'] = 57
test.loc[test.shop_id == 0, 'shop_id'] = 57
# Якутск ТЦ "Центральный"
train.loc[train.shop_id == 1, 'shop_id'] = 58
test.loc[test.shop_id == 1, 'shop_id'] = 58
# Жуковский ул. Чкалова 39м²
train.loc[train.shop_id == 10, 'shop_id'] = 11
test.loc[test.shop_id == 10, 'shop_id'] = 11

##### remove price and daily connt outliers ####

train = train[train.item_price<100000]
train = train[train.item_cnt_day<1001]

#### fill pirce below zero with meadian ####

median = train[(train.shop_id==32)&(train.item_id==2973)&(train.date_block_num==4)&(train.item_price>0)].item_price.median()
train.loc[train.item_price<0, 'item_price'] = median

### select those shop and items that appears in test. (think of ways to utilize those unpresanted later)

In [5]:
train_lk = train[train.shop_id.isin(test.shop_id.unique())]
train_lk = train_lk[train_lk.item_id.isin(test.item_id.unique())]
print('train_lk shape: ', train_lk.shape)

train_lk shape:  (1227789, 6)


### Add revenue

In [6]:
train_lk['revenue'] = train_lk['item_price'] * train_lk['item_cnt_day']

### generate month sum and month mean

In [7]:
train_lk_gp = train_lk.groupby(['date_block_num', 'shop_id', 'item_id'], as_index=False)
train_lk = train_lk_gp.agg({'item_cnt_day': ['sum', 'mean'], 'item_price': ['mean'], 'revenue': ['sum', 'mean']})
train_lk.columns = ['date_block_num', 'shop_id', 'item_id', 'item_cnt_month', 'item_cnt_mean', 'item_price_mean',
                   'revenue_sum', 'revenue_mean']

cnt_bound = train_lk.item_cnt_month.quantile(0.99)
mean_bound = train_lk.item_cnt_mean.quantile(0.99)
print('item_cnt_month 99% quantile: ', cnt_bound)
print('item_cnt_mean 99% quantile: ', mean_bound)
# clip item_cnt_month between [0, 21] and item_cnt_mean between [0, 2.5]
train_lk['item_cnt_month'] = train_lk['item_cnt_month'].clip(lower=0, upper=cnt_bound)
train_lk['item_cnt_mean'] = train_lk['item_cnt_mean'].clip(lower=0, upper=mean_bound)

item_cnt_month 99% quantile:  21.0
item_cnt_mean 99% quantile:  2.5


### Add item category

In [8]:
train_lk = pd.merge(train_lk, item[['item_id', 'item_category_id']], how='left', on='item_id')

In [9]:
train_lk.head(5)

,date_block_num,shop_id,item_id,item_cnt_month,item_cnt_mean,item_price_mean,revenue_sum,revenue_mean,item_category_id
0,0,2,33,1.0,1.0,499.0,499.0,499.0,37
1,0,2,482,1.0,1.0,3300.0,3300.0,3300.0,73
2,0,2,491,1.0,1.0,600.0,600.0,600.0,73
3,0,2,839,1.0,1.0,3300.0,3300.0,3300.0,73
4,0,2,1007,3.0,1.0,449.0,1347.0,449.0,67


### Generate cartisian products of date_block_num, shop_id, item_id

In [10]:
car_prod = np.array(list(product(*[train_lk.date_block_num.unique(), train_lk.shop_id.unique(), train_lk.item_id.unique()])))
train_car = pd.DataFrame(car_prod, columns=['date_block_num', 'shop_id', 'item_id'])
print('train_cartisian shape: ', train_car.shape)

train_lk = pd.merge(train_car, train_lk, how='left', on=['date_block_num', 'shop_id', 'item_id'])
train_lk.fillna(0, inplace=True)

train_cartisian shape:  (6734448, 3)


In [11]:
train_lk.head(2)

,date_block_num,shop_id,item_id,item_cnt_month,item_cnt_mean,item_price_mean,revenue_sum,revenue_mean,item_category_id
0,0,2,33,1.0,1.0,499.0,499.0,499.0,37.0
1,0,2,482,1.0,1.0,3300.0,3300.0,3300.0,73.0


### Add lag features

In [12]:
%%time
lags = [1, 2, 3]
lag_cols = ['item_cnt_month', 'item_cnt_mean']

for col in lag_cols:
    for lag in lags:
        train_lk[col +'_lag_' + str(lag)] = train_lk.sort_values('date_block_num')\
        .groupby(['shop_id', 'item_id'])[col].shift(lag)

CPU times: user 20 s, sys: 6.8 s, total: 26.8 s
Wall time: 15.6 s


In [13]:
%%time
train_lk.tail()

CPU times: user 762 µs, sys: 225 µs, total: 987 µs
Wall time: 854 µs


,date_block_num,shop_id,item_id,item_cnt_month,item_cnt_mean,item_price_mean,revenue_sum,revenue_mean,item_category_id,item_cnt_month_lag_1,item_cnt_month_lag_2,item_cnt_month_lag_3,item_cnt_mean_lag_1,item_cnt_mean_lag_2,item_cnt_mean_lag_3
6734443,33,36,9103,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6734444,33,36,9107,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6734445,33,36,10204,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6734446,33,36,12733,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6734447,33,36,15925,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Add price difference with previous months

In [14]:
lags = [1, 2]

for lag in lags:
    train_lk['item_price_mean' + '_lag_' + str(lag)] = train_lk.sort_values('date_block_num')\
    .groupby(['shop_id', 'item_id'])['item_price_mean'].diff(lag)

### Add rolling stats

In [15]:
%%time
window_size = 3

f_max = lambda x : x.rolling(window_size, min_periods=1).max()
f_min = lambda x : x.rolling(window_size, min_periods=1).min()
f_mean = lambda x : x.rolling(window_size, min_periods=1).mean()
f_std = lambda x: x.rolling(window_size, min_periods=1).std()

funcs = [f_max, f_min, f_mean, f_std]
func_names = ['_max', '_min', '_mean', '_std']

for func, func_name in zip(funcs, func_names):
    
    print('working on {} ...'.format(func_name))
    train_lk['item_cnt_month_' + str(window_size) + func_name] = train_lk.sort_values('date_block_num') \
    .groupby(['shop_id', 'item_id'])['item_cnt_month'].apply(func)

working on _max ...
working on _min ...
working on _mean ...
working on _std ...
CPU times: user 5min 27s, sys: 15.4 s, total: 5min 42s
Wall time: 5min 28s


In [16]:
train_lk.sample(5)

,date_block_num,shop_id,item_id,item_cnt_month,item_cnt_mean,item_price_mean,revenue_sum,revenue_mean,item_category_id,item_cnt_month_lag_1,...,item_cnt_month_lag_3,item_cnt_mean_lag_1,item_cnt_mean_lag_2,item_cnt_mean_lag_3,item_price_mean_lag_1,item_price_mean_lag_2,item_cnt_month_3_max,item_cnt_month_3_min,item_cnt_month_3_mean,item_cnt_month_3_std
5843246,29,38,17717,8.0,1.142857,1267.714286,10010.0,1430.0,79.0,18.0,...,21.0,1.2,1.166667,1.333333,198.047619,10.964286,18.0,8.0,13.333333,5.033223
3359982,16,34,19999,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000
1952161,9,5,3034,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000
5554959,28,3,13587,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000
453286,2,21,11365,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,NaN,0.0,0.000000,NaN,0.000000,0.000000,0.0,0.0,0.000000,0.000000


### Add month feature

In [17]:
train_lk['month'] = train_lk['date_block_num'] % 12

### Identify target

Since our job is to predict the next month salses, the target should be next month sales not the current month!

In [18]:
train_lk['target'] = train_lk.sort_values('date_block_num').groupby(['shop_id', 'item_id'])['item_cnt_month'].shift(-1)
train_lk.tail()

,date_block_num,shop_id,item_id,item_cnt_month,item_cnt_mean,item_price_mean,revenue_sum,revenue_mean,item_category_id,item_cnt_month_lag_1,...,item_cnt_mean_lag_2,item_cnt_mean_lag_3,item_price_mean_lag_1,item_price_mean_lag_2,item_cnt_month_3_max,item_cnt_month_3_min,item_cnt_month_3_mean,item_cnt_month_3_std,month,target
6734443,33,36,9103,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9,NaN
6734444,33,36,9107,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9,NaN
6734445,33,36,10204,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9,NaN
6734446,33,36,12733,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9,NaN
6734447,33,36,15925,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9,NaN


### Train, val, test splitting

In [19]:
df_train = train_lk[(train_lk.date_block_num > 2) & (train_lk.date_block_num < 30)]
df_val = train_lk[(train_lk.date_block_num >=30) & (train_lk.date_block_num <= 32)]
df_test = pd.merge(test, train_lk[train_lk.date_block_num == 33], how='left', on=['shop_id', 'item_id'])\
    .drop('target', axis=1)

### Mean encoding

In [21]:
%%time
enc_cols = ['shop_id', 'item_id', 'month', 'date_block_num']

#### first order mean encoding ####
for col in enc_cols:
    df_enc = df_train.groupby(col, as_index=False) \
    .agg({'target': 'mean'}).rename({'target': col + '_mean_enc'}, axis=1)
    
    df_train = df_train.merge(df_enc, on=col, how='left')
    df_val = df_val.merge(df_enc, on=col, how='left')
    df_test = df_test.merge(df_enc, on=col, how='left')
  

CPU times: user 16.7 s, sys: 11.6 s, total: 28.3 s
Wall time: 17.5 s


In [23]:
%%time
#### second order mean encoding ####
for col_pair in combinations(enc_cols, 2):
    pair_name = col_pair[0] + '_' + col_pair[1]
    df_enc = df_train.groupby(col_pair, as_index=False) \
    .agg({'target': 'mean'}).rename({'target': pair_name + '_mean_enc'}, axis=1)
    
    df_train = df_train.merge(df_enc, on=col_pair, how='left')
    df_val = df_val.merge(df_enc, on=col_pair, how='left')
    df_test = df_test.merge(df_enc, on=col_pair, how='left')

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: Interpreting tuple 'by' as a list of keys, rather than a single key. Use 'by=[...]' instead of 'by=(...)'. In the future, a tuple will always mean a single key.
  after removing the cwd from sys.path.


CPU times: user 33.7 s, sys: 20.8 s, total: 54.5 s
Wall time: 33.2 s


### Select correct columns

In [26]:
train_cols = df_train.columns.values.tolist()
train_cols.remove('target')

label_col = 'target'

x_train, y_train = df_train[train_cols], df_train[label_col]
x_val, y_val = df_val[train_cols], df_val[label_col]

### Fill NAs in validation with zeros

In [33]:
x_val.fillna(0, inplace=True)

/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3787: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


### Fill NAs in Test with zeros

In [27]:
x_test = df_test[train_cols]
x_test.fillna(0, inplace=True)

/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3787: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [34]:
x_test.isnull().sum()

date_block_num                     0
shop_id                            0
item_id                            0
item_cnt_month                     0
item_cnt_mean                      0
item_price_mean                    0
revenue_sum                        0
revenue_mean                       0
item_category_id                   0
item_cnt_month_lag_1               0
item_cnt_month_lag_2               0
item_cnt_month_lag_3               0
item_cnt_mean_lag_1                0
item_cnt_mean_lag_2                0
item_cnt_mean_lag_3                0
item_price_mean_lag_1              0
item_price_mean_lag_2              0
item_cnt_month_3_max               0
item_cnt_month_3_min               0
item_cnt_month_3_mean              0
item_cnt_month_3_std               0
month                              0
shop_id_mean_enc                   0
item_id_mean_enc                   0
month_mean_enc                     0
date_block_num_mean_enc            0
shop_id_item_id_mean_enc           0
s

In [35]:
x_val.isnull().sum()

date_block_num                     0
shop_id                            0
item_id                            0
item_cnt_month                     0
item_cnt_mean                      0
item_price_mean                    0
revenue_sum                        0
revenue_mean                       0
item_category_id                   0
item_cnt_month_lag_1               0
item_cnt_month_lag_2               0
item_cnt_month_lag_3               0
item_cnt_mean_lag_1                0
item_cnt_mean_lag_2                0
item_cnt_mean_lag_3                0
item_price_mean_lag_1              0
item_price_mean_lag_2              0
item_cnt_month_3_max               0
item_cnt_month_3_min               0
item_cnt_month_3_mean              0
item_cnt_month_3_std               0
month                              0
shop_id_mean_enc                   0
item_id_mean_enc                   0
month_mean_enc                     0
date_block_num_mean_enc            0
shop_id_item_id_mean_enc           0
s

### Gradient boosting tree

In [29]:
from sklearn.ensemble import GradientBoostingRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error

In [37]:
%%time
params = {
    'learning_rate': 0.1,
    'max_depth': 3,
    'n_estimators': 100,
    'subsample': 0.5,
    'random_state': 41
}


gbr = GradientBoostingRegressor(**params)
gbr.fit(x_train, y_train)

CPU times: user 14min 33s, sys: 13.4 s, total: 14min 46s
Wall time: 14min 46s


In [38]:
y_pred_train = gbr.predict(x_train)
print('train RMSE: ', np.sqrt(mean_squared_error(y_train, y_pred_train)))
y_pred_val = gbr.predict(x_val)
print('validation RMSE: ', np.sqrt(mean_squared_error(y_val, y_pred_val)))

train RMSE:  0.5268404038242678
validation RMSE:  1.0041617210518001


In [ ]:
# y_pred_test = gbr.predict(x_test)
# pred_test = test.copy()
# pred_test['item_cnt_month'] = y_pred_test
# save_pred_to_csv(pred_test, msg='three_month_lag-zero_fill_shop_item_month_mean_enc-two_month_price_lag-rolling-stats')

with outlier
train RMSE:  1.8482461723357235
validation RMSE:  3.116295303446713

without outlier
train RMSE:  0.8136265756274446
validation RMSE:  0.9271415425647744

with shop, item, month mean encoding
train RMSE:  0.783626998745065
validation RMSE:  0.9236206120907205

with shop, item, month mean encoding plus 2, 3 month lag
train RMSE:  0.7878361503843763
validation RMSE:  0.9178575199731875

with shop, item, month mean encoding plus 3 month lag, rolling stats, 1, 2 month price diff
train RMSE:  0.7841229329310745
validation RMSE:  0.9147349797087506